In [10]:
%%timeit
a = [1, 2, 3, 4, 5]
b = [5, 6, 14 ,7 ,8]
d = a + b
d

255 ns ± 3.57 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%%timeit
a = [1, 2, 3, 4, 5]
b = [5, 6, 14 ,7 ,8]
a.extend(b)
a

271 ns ± 6.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
%%timeit
a = [ ]
for i in range(10**6):
    a.append(i)

134 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
a = list(range(10**6))
for i in range(10**6):
    a[i] = i

107 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<b> Текст </b> 

In [ ]:
!pip install beautifulsoup4

In [19]:
import requests
from bs4 import BeautifulSoup

In [24]:
url = "https://knowyourmeme.com/"
requests.get(url)

<Response [403]>

In [22]:
url = "http://books.toscrape.com/catalogue/page-3.html666"
requests.get(url)

<Response [404]>

In [26]:
url = "http://books.toscrape.com/catalogue/page-3.html"
res = requests.get(url)
res

<Response [200]>

In [30]:
soup = BeautifulSoup(res.content)
soup.title.text

'\n    All products | Books to Scrape - Sandbox\n'

In [33]:
item = soup.find("article", {"class": "product_pod"})
item

<article class="product_pod">
<div class="image_container">
<a href="slow-states-of-collapse-poems_960/index.html"><img alt="Slow States of Collapse: Poems" class="thumbnail" src="../media/cache/72/41/72417db983862010ef0c1a25de98c7d7.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="slow-states-of-collapse-poems_960/index.html" title="Slow States of Collapse: Poems">Slow States of Collapse: ...</a></h3>
<div class="product_price">
<p class="price_color">£57.31</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [35]:
type(item)

bs4.element.Tag

In [37]:
item.h3.a

<a href="slow-states-of-collapse-poems_960/index.html" title="Slow States of Collapse: Poems">Slow States of Collapse: ...</a>

In [38]:
item.h3.a.text

'Slow States of Collapse: ...'

In [39]:
item.h3.a.get('href')

'slow-states-of-collapse-poems_960/index.html'

In [40]:
item.h3.a.get('title')

'Slow States of Collapse: Poems'

In [43]:
items = soup.find_all("article", {"class": "product_pod"})
[item.h3.a.get('href') for item in items]

['slow-states-of-collapse-poems_960/index.html',
 'reasons-to-stay-alive_959/index.html',
 'private-paris-private-10_958/index.html',
 'higherselfie-wake-up-your-life-free-your-soul-find-your-tribe_957/index.html',
 'without-borders-wanderlove-1_956/index.html',
 'when-we-collided_955/index.html',
 'we-love-you-charlie-freeman_954/index.html',
 'untitled-collection-sabbath-poems-2014_953/index.html',
 'unseen-city-the-majesty-of-pigeons-the-discreet-charm-of-snails-other-wonders-of-the-urban-wilderness_952/index.html',
 'unicorn-tracks_951/index.html',
 'unbound-how-eight-technologies-made-us-human-transformed-society-and-brought-our-world-to-the-brink_950/index.html',
 'tsubasa-world-chronicle-2-tsubasa-world-chronicle-2_949/index.html',
 'throwing-rocks-at-the-google-bus-how-growth-became-the-enemy-of-prosperity_948/index.html',
 'this-one-summer_947/index.html',
 'thirst_946/index.html',
 'the-torch-is-passed-a-harding-family-story_945/index.html',
 'the-secret-of-dreadwillow-carse_

In [45]:
def get_hrefs(p):
    url = f"http://books.toscrape.com/catalogue/page-{p}.html"
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    items = soup.find_all("article", {"class": "product_pod"})
    hrefs = [item.h3.a.get('href') for item in items]
    titles = [item.h3.a.get('title') for item in items]
    return titles, hrefs

In [47]:
from tqdm.notebook import tqdm 

titles = [ ]
hrefs = [ ]
for p in tqdm(range(1, 51)):
    t,h = get_hrefs(p)
    titles.extend(t)
    hrefs.extend(h)

100%|██████████| 50/50 [00:35<00:00,  1.40it/s]


In [49]:
import pandas as pd 
df = pd.DataFrame({'title': titles, 'href': hrefs})
df.head()

,title,href
0,A Light in the Attic,a-light-in-the-attic_1000/index.html
1,Tipping the Velvet,tipping-the-velvet_999/index.html
2,Soumission,soumission_998/index.html
3,Sharp Objects,sharp-objects_997/index.html
4,Sapiens: A Brief History of Humankind,sapiens-a-brief-history-of-humankind_996/index...


In [50]:
df.to_csv('hrefs.csv', sep='\t')

In [70]:
def get_info(href):
    main_url = "http://books.toscrape.com/catalogue/"
    main_url += href
    
    res = requests.get(main_url)
    soup = BeautifulSoup(res.content)
    
    # desc = soup.find('article').find_all('p')[-1].text
    price = soup.find('p', {"class" :"price_color"}).text
    
    info = {
        'href': href, 
        'describe': desc, 
        'price': price
    }
    return info

In [80]:
pd.read_html("http://books.toscrape.com/catalogue/sleeping-giants-themis-files-1_604/index.html")

[                   0                        1
 0                UPC         f435aa99d864077d
 1       Product Type                    Books
 2  Price (excl. tax)                   £48.74
 3  Price (incl. tax)                   £48.74
 4                Tax                    £0.00
 5       Availability  In stock (11 available)
 6  Number of reviews                        0]

In [78]:
pd.read_html(str(soup.find('table')))[0]

,0,1
0,UPC,a897fe39b1053632
1,Product Type,Books
2,Price (excl. tax),£51.77
3,Price (incl. tax),£51.77
4,Tax,£0.00
5,Availability,In stock (22 available)
6,Number of reviews,0


In [71]:
info = [ ]
for url in df.href.values[:10]:
    info.append(get_info(url))

In [72]:
df = pd.DataFrame(info)
df.head()

,href,describe,price
0,a-light-in-the-attic_1000/index.html,It's hard to imagine a world without A Light i...,£51.77
1,tipping-the-velvet_999/index.html,\n\n \n In stock\n \n,£53.74
2,soumission_998/index.html,\n\n \n In stock\n \n,£50.10
3,sharp-objects_997/index.html,\n\n \n In stock\n \n,£47.82
4,sapiens-a-brief-history-of-humankind_996/index...,\n\n \n In stock\n \n,£54.23
